In [2]:
# Import of libraries
import matplotlib.pyplot as plt
import plotly as py
import pandas as pd

In [3]:
# Load data
recdata = pd.read_excel('KPI Team.xlsx', sheet_name='Blad1', header=1)

In [4]:
# Initial data exploration
recdata.info()
recdata.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           37 non-null     object        
 1   Week           37 non-null     float64       
 2   Begin datum    37 non-null     datetime64[ns]
 3   Eind datum     37 non-null     datetime64[ns]
 4   Cold call      27 non-null     object        
 5   Qualification  31 non-null     object        
 6   Introductions  36 non-null     object        
 7   InMails        34 non-null     object        
 8   Response rate  31 non-null     object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 3.0+ KB


,Name,Week,Begin datum,Eind datum,Cold call,Qualification,Introductions,InMails,Response rate
0,Iris,32.0,2025-08-04,2025-08-08,22,9,3,165,0.28
1,Gianluca,32.0,2025-08-04,2025-08-08,NaN,8,4,34,0.34
2,Mariana,32.0,2025-08-04,2025-08-08,1,6,3,121,0.182
3,Agustina,32.0,2025-08-04,2025-08-08,0,6,2,55,0.44
4,Iris,33.0,2025-08-11,2025-08-15,11,14,3,157,0.48
5,Gianluca,33.0,2025-08-11,2025-08-15,0,3,2,1,0
6,Mariana,33.0,2025-08-11,2025-08-15,0,10,5,NaN,NaN
7,Agustina,33.0,2025-08-11,2025-08-15,0,6,1,70,0.2
8,Iris,34.0,2025-08-18,2025-08-22,8,7,3,87,0.31
9,Gianluca,34.0,2025-08-18,2025-08-22,holiday,holiday,holiday,holiday,holiday


In [5]:
# Removing erroneous rows
recdata = recdata.drop(index=[37, 38, 39, 40])

# Datetime conversion
recdata['Begin datum'] = pd.to_datetime(recdata['Begin datum'], format='%d/%m/%Y')
recdata['Eind datum'] = pd.to_datetime(recdata['Eind datum'], format='%d/%m/%Y')

# Extraction of week number
recdata['Week'] = recdata['Begin datum'].dt.isocalendar().week

# filtering and replacing nans
recdata = recdata.fillna(0)
recdata = recdata.replace('holiday', 0)


C:\Users\sanne\AppData\Local\Temp\ipykernel_23100\2891145356.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recdata = recdata.replace('holiday', 0)


In [6]:
# Converting columns to appropriate data types
str_cols = ['Name']
int_cols = ['Cold call', 'Qualification', 'Introductions', 'InMails']
float_cols = ['Response rate']

# Ensure 'Name' column is converted to string
recdata['Name'] = recdata['Name'].astype('string')

for col in int_cols:
    recdata[col] = recdata[col].astype(int)

# Clean and convert 'Response rate' to float (as fraction if '%' present)
for col in float_cols:
    recdata['Response rate'] = recdata['Response rate'].astype(str).apply(
        lambda x: float(x.replace('%', '')) / 100 if '%' in x else float(x)
)

In [7]:
recdata.head(10)

,Name,Week,Begin datum,Eind datum,Cold call,Qualification,Introductions,InMails,Response rate
0,Iris,32,2025-08-04,2025-08-08,22,9,3,165,0.280
1,Gianluca,32,2025-08-04,2025-08-08,0,8,4,34,0.340
2,Mariana,32,2025-08-04,2025-08-08,1,6,3,121,0.182
3,Agustina,32,2025-08-04,2025-08-08,0,6,2,55,0.440
4,Iris,33,2025-08-11,2025-08-15,11,14,3,157,0.480
5,Gianluca,33,2025-08-11,2025-08-15,0,3,2,1,0.000
6,Mariana,33,2025-08-11,2025-08-15,0,10,5,0,0.000
7,Agustina,33,2025-08-11,2025-08-15,0,6,1,70,0.200
8,Iris,34,2025-08-18,2025-08-22,8,7,3,87,0.310
9,Gianluca,34,2025-08-18,2025-08-22,0,0,0,0,0.000


In [8]:
# Calculate 'Time to hire' in days
# NOTE: Not actual 'Time to hire', likely just the employee's active worktime.
recdata['Time to hire'] = (recdata['Eind datum'] - recdata['Begin datum']).dt.days

recdata = recdata[['Name', 'Week', 'Begin datum', 'Eind datum', 'Time to hire', 'Cold call', 'InMails', 'Qualification', 'Introductions', 'Response rate']]

recdata.head()

,Name,Week,Begin datum,Eind datum,Time to hire,Cold call,InMails,Qualification,Introductions,Response rate
0,Iris,32,2025-08-04,2025-08-08,4,22,165,9,3,0.280
1,Gianluca,32,2025-08-04,2025-08-08,4,0,34,8,4,0.340
2,Mariana,32,2025-08-04,2025-08-08,4,1,121,6,3,0.182
3,Agustina,32,2025-08-04,2025-08-08,4,0,55,6,2,0.440
4,Iris,33,2025-08-11,2025-08-15,4,11,157,14,3,0.480


In [9]:
contact = recdata['InMails'] + recdata['Cold call']

recdata['Responses (accepted or declined)'] = ((recdata['InMails'] + recdata['Cold call']) * recdata['Response rate']).round().astype(int)

recdata = recdata[['Name', 'Week', 'Begin datum', 'Eind datum', 'Time to hire', 'Cold call', 'InMails', 'Qualification', 'Introductions', 'Responses (accepted or declined)', 'Response rate']]

recdata.head()

,Name,Week,Begin datum,Eind datum,Time to hire,Cold call,InMails,Qualification,Introductions,Responses (accepted or declined),Response rate
0,Iris,32,2025-08-04,2025-08-08,4,22,165,9,3,52,0.280
1,Gianluca,32,2025-08-04,2025-08-08,4,0,34,8,4,12,0.340
2,Mariana,32,2025-08-04,2025-08-08,4,1,121,6,3,22,0.182
3,Agustina,32,2025-08-04,2025-08-08,4,0,55,6,2,24,0.440
4,Iris,33,2025-08-11,2025-08-15,4,11,157,14,3,81,0.480


In [10]:
recdata['Introductions to first contact ratio (%)'] = recdata.apply(
    lambda row: round(row['Introductions'] / (row['InMails'] + row['Cold call']) * 100, 2)
    if (row['InMails'] + row['Cold call']) > 0
    else 0,
    axis=1,
)

recdata = recdata[['Name', 'Week', 'Begin datum', 'Eind datum', 'Time to hire', 'Cold call', 'InMails', 'Qualification', 'Introductions', 'Responses (accepted or declined)', 'Response rate', 'Introductions to first contact ratio (%)']]

recdata.head()

,Name,Week,Begin datum,Eind datum,Time to hire,Cold call,InMails,Qualification,Introductions,Responses (accepted or declined),Response rate,Introductions to first contact ratio (%)
0,Iris,32,2025-08-04,2025-08-08,4,22,165,9,3,52,0.280,1.60
1,Gianluca,32,2025-08-04,2025-08-08,4,0,34,8,4,12,0.340,11.76
2,Mariana,32,2025-08-04,2025-08-08,4,1,121,6,3,22,0.182,2.46
3,Agustina,32,2025-08-04,2025-08-08,4,0,55,6,2,24,0.440,3.64
4,Iris,33,2025-08-11,2025-08-15,4,11,157,14,3,81,0.480,1.79


In [11]:
# Based on the assumption that more than 3 introductions lead to one candidate getting through.
recdata['Candidate employment'] = recdata.apply(
    lambda row: 1 if row['Introductions'] > 3 else 0,
    axis=1,
)

recdata = recdata[['Name', 'Week', 'Begin datum', 'Eind datum', 'Time to hire', 'Cold call', 'InMails', 'Qualification', 'Introductions', 'Candidate employment', 'Responses (accepted or declined)', 'Response rate', 'Introductions to first contact ratio (%)']]

recdata.head(10)

,Name,Week,Begin datum,Eind datum,Time to hire,Cold call,InMails,Qualification,Introductions,Candidate employment,Responses (accepted or declined),Response rate,Introductions to first contact ratio (%)
0,Iris,32,2025-08-04,2025-08-08,4,22,165,9,3,0,52,0.280,1.60
1,Gianluca,32,2025-08-04,2025-08-08,4,0,34,8,4,1,12,0.340,11.76
2,Mariana,32,2025-08-04,2025-08-08,4,1,121,6,3,0,22,0.182,2.46
3,Agustina,32,2025-08-04,2025-08-08,4,0,55,6,2,0,24,0.440,3.64
4,Iris,33,2025-08-11,2025-08-15,4,11,157,14,3,0,81,0.480,1.79
5,Gianluca,33,2025-08-11,2025-08-15,4,0,1,3,2,0,0,0.000,200.00
6,Mariana,33,2025-08-11,2025-08-15,4,0,0,10,5,1,0,0.000,0.00
7,Agustina,33,2025-08-11,2025-08-15,4,0,70,6,1,0,14,0.200,1.43
8,Iris,34,2025-08-18,2025-08-22,4,8,87,7,3,0,29,0.310,3.16
9,Gianluca,34,2025-08-18,2025-08-22,4,0,0,0,0,0,0,0.000,0.00


In [16]:
import streamlit as st
import pandas as pd
import plotly.express as px

# ===== Data inladen =====
df = pd.read_excel("KPI Team.xlsx", header=2)  # Rij 3 bevat de kolomnamen

# Verwijder eventuele rijen zoals 'Eindtotaal'
df = df[df["Name"].str.lower() != "eindtotaal"]

# Kolomnamen opschonen
df.columns = df.columns.str.strip()

# ===== Gemiddelden berekenen =====
avg_inmails = recdata["InMails"].mean()
avg_coldcalls = recdata["Cold call"].mean()
avg_response = recdata["Response rate"].mean()

# Dataframe voor visualisatie van KPI's
kpi_data = pd.DataFrame({
    "KPI": ["InMails", "Cold call", "Response rate"],
    "Gemiddelde": [avg_inmails, avg_coldcalls, avg_response]
})

# ===== Streamlit Layout =====
st.set_page_config(page_title="Recruitment KPI Dashboard", layout="wide")
st.title("📊 Recruitment KPI Dashboard")

# Tabs voor Input & Output
tab1, tab2 = st.tabs(["Input KPI's", "Output KPI's"])

with tab1:
    st.header("Input KPI's")

    # Gemiddelde KPI's
    fig = px.bar(
        kpi_data,
        x="KPI",
        y="Gemiddelde",
        text_auto=".2f",
        title="Gemiddelde Input KPI's"
    )
    st.plotly_chart(fig, use_container_width=True)

    # Per recruiter breakdown
    col1, col2, col3 = st.columns(3)

    with col1:
        fig_inmails = px.bar(
            df,
            x="Name",
            y="Som van InMails",
            title="InMails per Recruiter"
        )
        st.plotly_chart(fig_inmails, use_container_width=True)

    with col2:
        fig_coldcalls = px.bar(
            recdata,
            x="Name",
            y="Cold call",
            title="Cold Calls per Recruiter"
        )
        st.plotly_chart(fig_coldcalls, use_container_width=True)

    with col3:
        fig_response = px.bar(
            df,
            x="Name",
            y="Gemiddelde van Response rate",
            title="Response Rate per Recruiter"
        )
        st.plotly_chart(fig_response, use_container_width=True)

with tab2:
    st.header("Output KPI's")
    st.info("Hier komen later de output KPI visualisaties 🚀")


2025-10-02 13:52:20.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-02 13:52:20.211 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar